<a href="https://colab.research.google.com/github/veiro/tesis-imputacion-datos/blob/main/codigo/imputadores/MIDASpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEER
1. Cambiar en enterno a GPU y la variable cuda= True
2. Para ejecutar todo el dataset, MODO_DESARROLLO=False, USAR_GOOGLE_DRIVE =  False y ejecutar un docker local de colab por que demora mas de 12 hrs.

## 1. Requirements

In [ ]:
USAR_GOOGLE_DRIVE =  True
MODO_DESARROLLO = False

In [ ]:
!pip install MIDASpy

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math

import MIDASpy as md

In [ ]:
if (USAR_GOOGLE_DRIVE):
  drive.mount('/content/gdrive')
  PATH_DATA_PROCESADA = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/data/data-procesada"
  PATH_IMPUTATION = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/imputacion/"
  sys.path.append('/content/gdrive/MyDrive/Maestria/Tesis/Codigo/utils')


else:
  PATH_DATA_PROCESADA = "/mnt"
  PATH_IMPUTATION = "/mnt/"

import utils as utils
import importlib
importlib.reload(utils)

if (30 ==  utils.version()):
  print("version correcta")
else:
  raise Exception("Version de Util vieja")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
version correcta


In [ ]:
!python3 -m pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.6.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.7
cuda-python==12.2.1
cudf-cu12 @ https://pypi.nvidia.c

## 3. Prepare Data

## Armo juego de datos entramiento y test


In [ ]:
df_train_encoded, df_train_encoded_missing, mask_train_open = utils.obtenerDatosTrain(MODO_DESARROLLO, PATH_DATA_PROCESADA)



In [ ]:
df_train_encoded.columns

Index(['Country_Region_Australia', 'Country_Region_Belgium',
       'Country_Region_Brazil', 'Country_Region_Canada',
       'Country_Region_Chile', 'Country_Region_China',
       'Country_Region_Colombia', 'Country_Region_Denmark',
       'Country_Region_France', 'Country_Region_Germany',
       'Country_Region_India', 'Country_Region_Italy', 'Country_Region_Japan',
       'Country_Region_Malaysia', 'Country_Region_Mexico',
       'Country_Region_Netherlands', 'Country_Region_New Zealand',
       'Country_Region_Pakistan', 'Country_Region_Peru',
       'Country_Region_Russia', 'Country_Region_Spain',
       'Country_Region_Sweden', 'Country_Region_US', 'Country_Region_Ukraine',
       'Country_Region_United Kingdom', 'Lat', 'Long_', 'Confirmed', 'Deaths',
       'Incident_Rate', 'Case_Fatality_Ratio', 'Province_State_encoded',
       'Last_Update_encoded', 'date_encoded'],
      dtype='object')

In [ ]:
def getData():
  categorical = ['Country_Region_Australia', 'Country_Region_Belgium',
        'Country_Region_Brazil', 'Country_Region_Canada',
        'Country_Region_Chile', 'Country_Region_China',
        'Country_Region_Colombia', 'Country_Region_Denmark',
        'Country_Region_France', 'Country_Region_Germany',
        'Country_Region_India', 'Country_Region_Italy', 'Country_Region_Japan',
        'Country_Region_Malaysia', 'Country_Region_Mexico',
        'Country_Region_Netherlands', 'Country_Region_New Zealand',
        'Country_Region_Pakistan', 'Country_Region_Peru',
        'Country_Region_Russia', 'Country_Region_Spain',
        'Country_Region_Sweden', 'Country_Region_US', 'Country_Region_Ukraine',
        'Country_Region_United Kingdom', 'date_encoded', 'Province_State_encoded']
  categorical = []
  data_cat, cat_cols_list = md.cat_conv(df_train_encoded_missing[categorical])

  df_train_encoded_missing.drop(categorical, axis = 1, inplace = True)
  constructor_list = [df_train_encoded_missing]
  constructor_list.append(data_cat)
  data_in = pd.concat(constructor_list, axis=1)

  na_loc = data_in.isnull()
  data_in[na_loc] = np.nan
  del na_loc
  del data_cat
  del constructor_list
  del categorical
  return data_in, cat_cols_list

#data_in, cat_cols_list = getData()

In [ ]:
def getDataTransformer(df):
  cat_cols_list = []
  data_in = df

  na_loc = data_in.isnull()
  data_in[na_loc] = np.nan

  scaler = MinMaxScaler()
  data_scaled = scaler.fit_transform(data_in)
  data_scaled = pd.DataFrame(data_scaled, columns = data_in.columns)
  return data_scaled, cat_cols_list, scaler

data_in,cat_cols_list, scaler = getDataTransformer(df_train_encoded_missing)

In [ ]:
data_in

,Country_Region_Australia,Country_Region_Belgium,Country_Region_Brazil,Country_Region_Canada,Country_Region_Chile,Country_Region_China,Country_Region_Colombia,Country_Region_Denmark,Country_Region_France,Country_Region_Germany,...,Country_Region_United Kingdom,Lat,Long_,Confirmed,Deaths,Incident_Rate,Case_Fatality_Ratio,Province_State_encoded,Last_Update_encoded,date_encoded
0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.222387,0.000009,0.000011,0.062400,NaN,0.861252,NaN,0.631117
1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.801800,0.220870,0.000018,0.000027,0.056040,0.072944,NaN,0.984204,0.209536
2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,NaN,0.290721,0.000572,0.000827,NaN,0.071834,NaN,0.614824,0.390213
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.780718,0.308854,0.000154,0.000333,0.015953,0.107368,0.494078,0.147023,0.376412
4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.751321,NaN,0.008881,0.035372,0.037621,0.197642,NaN,0.201701,0.486826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094551,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.351032,0.406098,0.043261,0.114840,0.027193,0.131734,NaN,0.569866,NaN
2094552,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,...,0.0,0.739064,0.210544,0.004619,0.002514,NaN,0.027014,0.187817,0.788578,0.746550
2094553,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.275588,0.021440,0.021522,0.087503,0.049813,0.238579,NaN,NaN
2094554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.214033,0.000254,0.000731,0.083594,0.142558,0.187817,0.927096,0.032622


## 4. Define Model

## 6. Train Model

In [ ]:
%%time
imputer = md.Midas(layer_structure = [256,256], vae_layer = False, seed = 89, input_drop = 0.75, savepath = PATH_IMPUTATION + '/modelo/MDIASpy_Desarrollo=' + str(MODO_DESARROLLO) +'/')
imputer.build_model(data_in, softmax_columns = cat_cols_list)
imputer.train_model(training_epochs = 20)

Size index: [34]

Computation graph constructed

Model initialised

Epoch: 0 , loss: 1.2672452051898684
Epoch: 1 , loss: 1.2015682394707945
Epoch: 2 , loss: 1.173475017009627
Epoch: 3 , loss: 1.1628399756851273
Epoch: 4 , loss: 1.1491394436386997
Epoch: 5 , loss: 1.1459696976521423
Epoch: 6 , loss: 1.1445223959808737
Epoch: 7 , loss: 1.1437365928605143
Epoch: 8 , loss: 1.1432449277220746
Epoch: 9 , loss: 1.1428381304280073
Epoch: 10 , loss: 1.1350588224534168
Epoch: 11 , loss: 1.1133506975640082
Epoch: 12 , loss: 1.1096774265701106
Epoch: 13 , loss: 1.107423950266458
Epoch: 14 , loss: 1.1051542501951286
Epoch: 15 , loss: 1.103228215187004
Epoch: 16 , loss: 1.1017639602687808
Epoch: 17 , loss: 1.1004992290063356
Epoch: 18 , loss: 1.0995307492333724
Epoch: 19 , loss: 1.0993559455396054
Training complete. Saving file...
Model saved in file: /content/gdrive/MyDrive/Maestria/Tesis/Codigo/imputacion//modelo/MDIASpy_Desarrollo=False/
CPU times: user 4h 56min 39s, sys: 1h 4min 21s, total: 6h 1

In [ ]:
imputations = imputer.generate_samples(m=10).output_list

Model restored.


## Cargo el modelo

## 7. Test Model

## armo datos test

In [ ]:
def get_mean_imputed(lista_dataframes):
  df_concatenado = pd.concat(lista_dataframes)

  # Calcular el promedio
  df_promedio = df_concatenado.groupby(level=0).mean()

  return df_promedio

In [ ]:
df_test_encoded, df_test_encoded_missing, mask_test_open = utils.obtenerDatosTest(MODO_DESARROLLO, PATH_DATA_PROCESADA)

df_test_encoded_missing = pd.DataFrame(df_test_encoded_missing, columns = df_train_encoded.columns)

In [ ]:
data_test, cols, scaler = getDataTransformer(df_test_encoded_missing)

In [ ]:
imputer_test = md.Midas(layer_structure = [256,256], vae_layer = False, seed = 89, input_drop = 0.75, savepath = PATH_IMPUTATION + '/modelo/MDIASpy_Desarrollo=' + str(MODO_DESARROLLO) +'/')
imputer_test.build_model(data_in, softmax_columns = cat_cols_list)

Size index: [34]

Computation graph constructed



In [ ]:

imputer_test.change_imputation_target(data_test)

In [ ]:
imputations = imputer_test.generate_samples(m=10).output_list
imputed_data = get_mean_imputed(imputations)
imputed_data = scaler.inverse_transform(imputed_data)

rmse = utils.rmse(df_test_encoded, imputed_data, mask_test_open)
print('rmse', rmse)


In [ ]:

imputed_data = pd.DataFrame(imputed_data, columns = df_train_encoded_missing.columns)
df_test_encoded = pd.DataFrame(df_test_encoded, columns = df_train_encoded_missing.columns)
imputed_data.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_MIDASpy_full_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)
df_test_encoded, imputed_data = utils.obtener_df_imputados(df_test_encoded, imputed_data, mask_test_open)

In [ ]:
print("inicio de decoding")
imputed_data_decoded= utils.decode(imputed_data, df_train_encoded, PATH_DATA_PROCESADA)


inicio de decoding


897667it [02:04, 7216.94it/s]


In [ ]:


print("imputed_data_decoded.head(5)")
print(imputed_data_decoded.head(5))

print("imputed_data_decoded.to_csv")
imputed_data_decoded.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_MIDASpy_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)

print("fin de decoding")

imputed_data_decoded.head(5)
  Province_State Country_Region          Last_Update        Lat    Long_  \
0          Texas             US  2021-01-02 05:22:33  41.123513 -83.1278   
1          Texas             US  2021-01-02 05:22:33  35.650623 -75.2996   
2          Texas             US  2021-01-02 05:22:33  35.780773 -89.8185   
3          Texas             US  2021-01-02 05:22:33  33.618867 -96.1364   
4          Texas             US  2021-01-02 05:22:33  43.720997 -87.9458   

   Confirmed  Deaths  Incident_Rate  Case_Fatality_Ratio        date  
0      12750     230   23107.035413             1.788235  01-01-2021  
1      10041     167   19549.853001             1.663181  01-01-2021  
2       4511     406   30784.732824             1.325795  01-01-2021  
3        543      27    4167.013225             5.389222  01-01-2021  
4      15026     171   13027.570661             1.138027  01-01-2021  
imputed_data_decoded.to_csv
fin de decoding


In [ ]:

target_variable = 'Country_Region'

class_counts = imputed_data_decoded[target_variable].value_counts().count()

print(class_counts)

25
